In [2]:
import requests

# 1. Setup the specific species and region
# 'indbun' is the code for Indigo Bunting
# 'US' is a good region to test since Indigo Buntings are common there
species_code = "indbun"
region_code = "US" 
url = f"https://api.ebird.org/v2/data/obs/{region_code}/recent/{species_code}"

headers = {'X-eBirdApiToken': 'tmb363lktsir'}

# 2. Execute the request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    if len(data) > 0:
        first_record = data[0]
        print(f"Total {species_code} records found: {len(data)}")
        print("\nAvailable 'Columns' (Keys) for Indigo Bunting:")
        
        # This loop prints every available field (column) for this species
        for key in first_record.keys():
            print(f"- {key}")
            
        # Optional: Print the first record's data to see example values
        # print("\nExample Data:", first_record)
    else:
        print(f"No recent sightings of {species_code} found in {region_code}.")
else:
    print(f"Error: {response.status_code}. Check your API key or URL.")

Total indbun records found: 106

Available 'Columns' (Keys) for Indigo Bunting:
- speciesCode
- comName
- sciName
- locId
- locName
- obsDt
- howMany
- lat
- lng
- obsValid
- obsReviewed
- locationPrivate
- subId


In [3]:
import requests
import pandas as pd

# 1. Setup API Details
species_code = "indbun"  # Indigo Bunting
region_code = "US"      
api_key = "tmb363lktsir"

# The '?back=30' parameter gets the last month of data
url = f"https://api.ebird.org/v2/data/obs/{region_code}/recent/{species_code}?back=30"
headers = {'X-eBirdApiToken': api_key}

# 2. Fetch the Data
print(f"Fetching data for {species_code}...")
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    
    if len(data) > 0:
        # 3. Convert JSON to a Pandas DataFrame
        df = pd.DataFrame(data)
        
        # 4. Save to CSV
        # 'index=False' prevents Pandas from adding an extra column for row numbers
        file_name = f"{species_code}_migration_data_last_30_days.csv"
        df.to_csv(file_name, index=False)
        
        print(f"Success! {len(df)} records saved to {file_name}")
        
        # Display the first few rows and all available columns
        print("\nPreview of saved columns:")
        print(df.columns.tolist())
    else:
        print("No sightings found in the last 30 days.")
else:
    print(f"Error {response.status_code}: {response.text}")

Fetching data for indbun...


KeyboardInterrupt: 

In [5]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# --- Setup ---
api_key = "tmb363lktsir"
region_code = "US"
species_code = "indbun"
headers = {'X-eBirdApiToken': api_key}

start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
all_observations = []
current_date = start_date

print(f"Starting data fetch for {species_code}...")

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y/%m/%d")
    url = f"https://api.ebird.org/v2/data/obs/{region_code}/historic/{formatted_date}"
    
    try:
        # TIMEOUT: If the server doesn't respond in 10 seconds, stop waiting
        response = requests.get(url, headers=headers)#, timeout=10)
        
        # This raises an error for 4xx or 5xx responses automatically
        response.raise_for_status()

        day_data = response.json()
        species_only = [obs for obs in day_data if obs['speciesCode'] == species_code]
        all_observations.extend(species_only)
        print(f"[{formatted_date}] Success: {len(species_only)} sightings.")

    except requests.exceptions.ConnectionError:
        print(f"[{formatted_date}] MACHINE ERROR: No internet connection.")
        break  # Stop the loop if you're offline
        
    except requests.exceptions.HTTPError as err:
        print(f"[{formatted_date}] SERVER ERROR: {err}")
        # If it's a 429 (Rate Limit), wait longer
        if response.status_code == 429:
            print("Being rate limited. Waiting 30 seconds...")
            time.sleep(30)
            continue
            
    except Exception as e:
        print(f"[{formatted_date}] UNEXPECTED ERROR: {e}")

    current_date += timedelta(days=1)
    time.sleep(0.5)

# --- Final Save ---
if all_observations:
    df = pd.DataFrame(all_observations)
    df.to_csv(f"{species_code}_2018_data.csv", index=False)
    print(f"\nDone! Saved {len(df)} total records.")

Starting data fetch for indbun...
[2018/01/01] SERVER ERROR: 500 Server Error: Internal Server Error for url: https://api.ebird.org/v2/data/obs/US/historic/2018/01/01
[2018/01/02] SERVER ERROR: 500 Server Error: Internal Server Error for url: https://api.ebird.org/v2/data/obs/US/historic/2018/01/02


KeyboardInterrupt: 